In [1]:
import pandas as pd
df = pd.read_csv('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Text Normalization\\Tweets.csv')

In [2]:
df

,Unnamed: 0,Tweet,Location,year,month,day
0,0,الفرق بين عروض فودافون وعروض اورانج عندي,egypt,2022,3,26
1,1,عروض باقات فودافون نت الشهرية لعام,egypt cairo,2022,3,25
2,2,سعر الراوتر الهوائي من فودافون بدون خط ارضي,Egypt,2022,3,25
3,3,لعملاء فودافون كان في كود بنعمله على الموبايل ...,"Cairo , Egypt",2022,3,23
4,4,شحن رصيد من فودافون كاش,Egypt,2022,3,23
...,...,...,...,...,...,...
877,877,و ممكن تجربة خطوات تنشيط الشبكة من خلال ضبط ا...,"Cairo, Egypt",2022,3,18
878,878,أهلا بيك وشكرا لتواصلك معانا ممكن توضيح اكتر...,"Cairo, Egypt",2022,3,18
879,879,فودافون بعتالي مبروك كسبت وحده مش مكفيهم ال...,Giza,2022,3,18
880,880,بعمل سكس فون و كاميرا ب تحويل فودافون كاش,مصر,2022,3,18


In [3]:
#df['Location'].fillna(value=" ", inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  882 non-null    int64 
 1   Tweet       882 non-null    object
 2   Location    882 non-null    object
 3   year        882 non-null    int64 
 4   month       882 non-null    int64 
 5   day         882 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 41.5+ KB


In [5]:
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-26e57-85547b8418c0.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [6]:
df.to_dict()

{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

In [7]:
tweets= db.collection(u'test').document(u'tweet').set(json.loads(json.dumps(df.to_dict())))